Data exploration and analysis
===================

Here we are going to explore the SBB dataset _is-daten-sbb_. We are going to select the features that can be of use for the rest of the project, clean the dataset if it needs to be and then compute some statistics on it.

## Imports

In [20]:
%matplotlib inline
import pandas as pd
print('pandas: {}'.format(pd.__version__))
import numpy as np
print('numpy: {}'.format(np.__version__))
import seaborn as sns
print('seaborn: {}'.format(sns.__version__))
import geopy as geo
print('geopy: {}'.format(geo.__version__))
import datetime

import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import powerlaw
from geopy import distance
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import normalize
from statistics import mean, median, stdev

pandas: 0.24.2
numpy: 1.16.4
seaborn: 0.9.0
geopy: 1.20.0


## Extraction

In [21]:
filepath='./data/ist-daten-sbb.csv'
df=pd.read_csv(filepath,delimiter=';')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61716 entries, 0 to 61715
Data columns (total 26 columns):
BETRIEBSTAG            61716 non-null object
FAHRT_BEZEICHNER       61716 non-null object
BETREIBER_ID           61716 non-null object
BETREIBER_ABK          61716 non-null object
BETREIBER_NAME         61716 non-null object
PRODUKT_ID             61636 non-null object
LINIEN_ID              61716 non-null int64
LINIEN_TEXT            61716 non-null object
UMLAUF_ID              0 non-null float64
VERKEHRSMITTEL_TEXT    61716 non-null object
ZUSATZFAHRT_TF         61716 non-null bool
FAELLT_AUS_TF          61716 non-null bool
BPUIC                  61716 non-null int64
HALTESTELLEN_NAME      61716 non-null object
ANKUNFTSZEIT           56478 non-null object
AN_PROGNOSE            52344 non-null object
AN_PROGNOSE_STATUS     61716 non-null object
ABFAHRTSZEIT           56477 non-null object
AB_PROGNOSE            52343 non-null object
AB_PROGNOSE_STATUS     61716 non-null object


The dataset contains 26 columns which will be not of use to us. we will now discuss the meaning of these columns and translate them.
The rows correspond to an arrival and departure of a train from one station.

| Column |     Description     | New name/drop |
| :---: | :--- | :--- |
| BETRIEBSTAG | is the day the train departed but there is some inconsistencies as some trains departs after midnight in the day after |departure_day |
|  FAHRT_BEZEICHNER | is an unique indicator for a given train which doesn't seem that useful since LINIEN_ID does that also | Dropped |
|  BETREIBER_ID | is the id of the operator of the train which is always SBB in this dataset | Dropped |
|  BETREIBER_ABK | is the abrevation of the operator as above it is always SBB | Dropped |
|  BETREIBER_NAME | is the full name of the operator | Dropped |
|  PRODUKT_ID | is an id that tells us the type of transport it is. It's always trains apart from a few missing values that should also be trains | product_id |
|  LINIEN_ID | is an id for a given course | course_id |
|  LINIEN_TEXT | is the type of course | transport_type |
|  UMLAUF_ID | is an ID that tells us if there as been any changes in the programmation. It is always null | Dropped |
|  VERKEHRSMITTEL_TEXT | is just LINIEN_TEXT without the number at the end | Dropped |
|  ZUSATZFAHRT_TF | is a boolean that tells us if the train was not planned ahead | planned |
|  FAELLT_AUS_TF | is a boolean that tells us if the train was down | down |
|  BPUIC | is the service number | BPUIC |
|  HALTESTELLEN_NAME | is the name of the station | station_name |
|  ANKUNFTSZEIT | is the planned arrival time of the train at that station |planned_arrival_time|
|  AN_PROGNOSE | is the actual arrival time of the train | actual_arrival_time |
|  AN_PROGNOSE_STATUS | is an indicator of the deletion of the arrival of the train | cancelled_arrival |
|  ABFAHRTSZEIT | is the planned departure time of the train |planned_departure_time|
|  AB_PROGNOSE | is the actual departure time of the train |actual_departure_time|
|  AB_PROGNOSE_STATUS | is an indicator of the deletion of the departure of the train |cancelled_departure |
|  DURCHFAHRT_TF | don't know what it is but it is always false | Dropped |
|  ankunftsverspatung | is a boolean that tells us if the arrival of the train was delayed |arrival_delayed|
|  abfahrtsverspatung | is a boolean that tells us if the departure of the train was delayed |departure_delayed|
|  lod | is a link to a webpage that tells us more information about station | Dropped |
|  geopos | is the geoposition of the station | geopos |
|  GdeNummer | don't know | Dropped |

## Data preprocessing
We are going to drop some columns and rename the remaining ones.


In [23]:
df.drop(['lod','GdeNummer','DURCHFAHRT_TF','UMLAUF_ID','BETREIBER_NAME','BETREIBER_ABK','BETREIBER_ID','FAHRT_BEZEICHNER','VERKEHRSMITTEL_TEXT'],axis=1,inplace=True)
df.columns=['departure_day','product_id','course_id','transport_type','planned','down','BPUIC','station_name','planned_arrival_time',\
          'actual_arrival_time','cancelled_arrival','planned_departure_time','actual_departure_time','cancelled_departure','arrival_delayed','departure_delayed','geopos']
df.sample(5)

,departure_day,product_id,course_id,transport_type,planned,down,BPUIC,station_name,planned_arrival_time,actual_arrival_time,cancelled_arrival,planned_departure_time,actual_departure_time,cancelled_departure,arrival_delayed,departure_delayed,geopos
61001,2019-11-02,Zug,19640,S16,False,False,8503004,Zürich Tiefenbrunnen,2019-11-02T11:07:00,2019-11-02T11:06:30,REAL,2019-11-02T11:07:00,2019-11-02T11:07:56,REAL,False,False,"47.3501235091,8.56137239898"
12799,2019-11-02,Zug,11658,S6,False,False,8503227,Näfels-Mollis,2019-11-02T14:50:00,2019-11-02T14:48:29,REAL,2019-11-02T14:50:00,2019-11-02T14:50:53,REAL,False,False,"47.0987196928,9.0699449253"
51817,2019-11-02,Zug,7837,R,False,False,8500207,Solothurn,2019-11-02T20:48:00,2019-11-02T20:48:28,REAL,2019-11-02T20:49:00,2019-11-02T20:49:59,REAL,False,False,"47.2041807751,7.54268420304"
32892,2019-11-02,Zug,25675,S20,False,False,8505213,Bellinzona,2019-11-02T19:24:00,2019-11-02T19:24:21,REAL,2019-11-02T19:32:00,2019-11-02T19:36:28,REAL,False,True,"46.195436816,9.02950898687"
25664,2019-11-02,Zug,2513,IR15,False,False,8504014,Palézieux,2019-11-02T08:59:00,2019-11-02T09:00:39,REAL,2019-11-02T09:00:00,2019-11-02T09:02:27,REAL,False,False,"46.5427523751,6.83787768542"


### Type conversion

We casted the date columns from string to datetime. We also considered to cast *geopos* but it's not required to use `geopy`.

In [24]:
df['planned_arrival_time'] =pd.to_datetime(df['planned_arrival_time'])
df['actual_arrival_time'] =pd.to_datetime(df['actual_arrival_time'])
df['planned_departure_time'] =pd.to_datetime(df['planned_departure_time'])
df['actual_departure_time'] =pd.to_datetime(df['actual_departure_time'])

In [25]:
location1 = df.loc[df['course_id'] == 30060].iloc[0]['geopos']
location2 = df.loc[df['course_id'] == 30060].iloc[1]['geopos']
#Excepted distance: 8 km (source: https://www.lexilogos.com/calcul_distances.htm)
print('location 1: ({}), location 2: ({}), distance: {} km'
      .format(location1, location2, round(geo.distance.distance(location1, location2).km),2))

location 1: (46.3983659275,6.92699984758), location 2: (46.4554432937,6.85953018827), distance: 8 km


In [26]:
#Translation of transport
df['product_id'].replace('Zug', 'train', inplace=True)

In [27]:
#Analyze the NaT values for arrival and departure time
df.loc[df['course_id'] == 30060].sort_values(by='planned_departure_time')

,departure_day,product_id,course_id,transport_type,planned,down,BPUIC,station_name,planned_arrival_time,actual_arrival_time,cancelled_arrival,planned_departure_time,actual_departure_time,cancelled_departure,arrival_delayed,departure_delayed,geopos
10497,2019-11-02,train,30060,S3,True,False,8501303,Villeneuve,NaT,NaT,PROGNOSE,2019-11-02 15:52:00,2019-11-02 15:52:54,REAL,False,False,"46.3983659275,6.92699984758"
22493,2019-11-02,train,30060,S3,True,False,8501300,Montreux,2019-11-02 15:56:00,2019-11-02 15:56:18,REAL,2019-11-02 15:56:00,2019-11-02 15:57:32,REAL,False,False,"46.4358743527,6.91043275261"
34234,2019-11-02,train,30060,S3,True,False,8501203,Clarens,2019-11-02 15:57:00,2019-11-02 15:58:22,REAL,2019-11-02 15:57:00,2019-11-02 15:59:37,REAL,False,False,"46.442731817,6.895777987"
22495,2019-11-02,train,30060,S3,True,False,8501202,Burier,2019-11-02 15:59:00,2019-11-02 16:02:01,REAL,2019-11-02 15:59:00,2019-11-02 16:02:50,REAL,True,True,"46.4478763825,6.87711567196"
10498,2019-11-02,train,30060,S3,True,False,8501201,La Tour-de-Peilz,2019-11-02 16:01:00,2019-11-02 16:04:20,REAL,2019-11-02 16:01:00,2019-11-02 16:04:51,REAL,True,True,"46.4554432937,6.85953018827"
22497,2019-11-02,train,30060,S3,True,False,8501200,Vevey,2019-11-02 16:07:00,2019-11-02 16:05:31,REAL,2019-11-02 16:07:00,2019-11-02 16:07:39,REAL,False,False,"46.4629974303,6.84345000982"
10499,2019-11-02,train,30060,S3,True,False,8501124,Cully,2019-11-02 16:14:00,2019-11-02 16:13:58,REAL,2019-11-02 16:14:00,2019-11-02 16:14:56,REAL,False,False,"46.488550014,6.72714757327"
33245,2019-11-02,train,30060,S3,True,False,8501122,Lutry,2019-11-02 16:17:00,2019-11-02 16:17:17,REAL,2019-11-02 16:17:00,2019-11-02 16:18:26,REAL,False,False,"46.5037789699,6.69061164399"
22499,2019-11-02,train,30060,S3,True,False,8501121,Pully,2019-11-02 16:20:00,2019-11-02 16:19:43,REAL,2019-11-02 16:20:00,2019-11-02 16:21:16,REAL,False,False,"46.5109189922,6.65946254087"
22501,2019-11-02,train,30060,S3,True,False,8501120,Lausanne,2019-11-02 16:24:00,2019-11-02 16:23:28,REAL,2019-11-02 16:33:00,2019-11-02 16:33:32,REAL,False,False,"46.5167786487,6.62909314109"


That shows it's normal to have NaT values for the date fields since the start point of the train has no arrival time and the arrival station has no planned departure.

Now looking at the cancelled departure we remark that it takes value PROGNOSE when it has no departure or arrival time which could either mean that it is the start of the line, the end of the line or that the train was really cancelled.

In [28]:
df.loc[df['course_id'] == 3260].sort_values(by='planned_departure_time')

,departure_day,product_id,course_id,transport_type,planned,down,BPUIC,station_name,planned_arrival_time,actual_arrival_time,cancelled_arrival,planned_departure_time,actual_departure_time,cancelled_departure,arrival_delayed,departure_delayed,geopos
35127,2019-11-02,train,3260,IR13,False,False,8509000,Chur,NaT,NaT,PROGNOSE,2019-11-02 07:12:00,NaT,PROGNOSE,False,False,"46.8530797552,9.52892561961"
20804,2019-11-02,train,3260,IR13,False,False,8509002,Landquart,2019-11-02 07:20:00,NaT,PROGNOSE,2019-11-02 07:22:00,NaT,PROGNOSE,False,False,"46.9674389764,9.55402859224"
23547,2019-11-02,train,3260,IR13,False,False,8509004,Bad Ragaz,2019-11-02 07:26:00,NaT,PROGNOSE,2019-11-02 07:27:00,NaT,PROGNOSE,False,False,"47.0103066077,9.50521579767"
23550,2019-11-02,train,3260,IR13,False,False,8509411,Sargans,2019-11-02 07:32:00,NaT,PROGNOSE,2019-11-02 07:36:00,2019-11-02 07:39:32,REAL,False,True,"47.0453635269,9.44539562754"
20806,2019-11-02,train,3260,IR13,False,False,8509404,Buchs SG,2019-11-02 07:45:00,2019-11-02 07:47:59,REAL,2019-11-02 07:45:00,2019-11-02 07:48:54,REAL,False,True,"47.1684089962,9.47862968189"
10899,2019-11-02,train,3260,IR13,False,False,8506319,Altstätten SG,2019-11-02 08:00:00,2019-11-02 08:02:28,REAL,2019-11-02 08:01:00,2019-11-02 08:03:23,REAL,False,False,"47.3742291734,9.55651096481"
34011,2019-11-02,train,3260,IR13,False,False,8506317,Heerbrugg,2019-11-02 08:06:00,2019-11-02 08:07:31,REAL,2019-11-02 08:07:00,2019-11-02 08:08:52,REAL,False,False,"47.4103853071,9.62724352946"
34013,2019-11-02,train,3260,IR13,False,False,8506314,St. Margrethen,2019-11-02 08:13:00,2019-11-02 08:12:50,REAL,2019-11-02 08:13:00,2019-11-02 08:13:41,REAL,False,False,"47.4532118157,9.63825154068"
23553,2019-11-02,train,3260,IR13,False,False,8506311,Rorschach,2019-11-02 08:21:00,2019-11-02 08:20:23,REAL,2019-11-02 08:21:00,2019-11-02 08:23:09,REAL,False,False,"47.4779984016,9.50513168035"
35130,2019-11-02,train,3260,IR13,False,False,8506302,St. Gallen,2019-11-02 08:35:00,2019-11-02 08:35:10,REAL,2019-11-02 08:37:00,NaT,PROGNOSE,False,False,"47.4231790668,9.36987712043"


What we see from this sample is that if the train is really cancelled the it has a planned arrival/departure time but not an actual. So we will change the values from cancelled arrival/departure to *Cancelled, Startpoint, Endpoint and somethingtosaynoproblem*.

In [38]:
df.at[(df.planned_arrival_time.isnull()) & (df.actual_arrival_time.isnull()),"cancelled_arrival"]="Startpoint"
df.at[(~df.planned_arrival_time.isnull()) & (df.actual_arrival_time.isnull()),"cancelled_arrival"]="Cancelled"
df.at[(~df.planned_arrival_time.isnull()) & (~df.actual_arrival_time.isnull()),"cancelled_arrival"]="Okay"
df.at[(df.planned_departure_time.isnull()) & (df.actual_departure_time.isnull()),"cancelled_departure"]="Endpoint"
df.at[(~df.planned_departure_time.isnull()) & (df.actual_departure_time.isnull()),"cancelled_departure"]="Cancelled"
df.at[(~df.planned_departure_time.isnull()) & (~df.actual_departure_time.isnull()),"cancelled_departure"]="Okay"

In [39]:
df.loc[df['course_id'] == 3260].sort_values(by='planned_departure_time')

,departure_day,product_id,course_id,transport_type,planned,down,BPUIC,station_name,planned_arrival_time,actual_arrival_time,cancelled_arrival,planned_departure_time,actual_departure_time,cancelled_departure,arrival_delayed,departure_delayed,geopos,departure_delay,arrival_delay
35127,2019-11-02,train,3260,IR13,False,False,8509000,Chur,NaT,NaT,Startpoint,2019-11-02 07:12:00,NaT,Cancelled,False,False,"46.8530797552,9.52892561961",00:00:00,00:00:00
20804,2019-11-02,train,3260,IR13,False,False,8509002,Landquart,2019-11-02 07:20:00,NaT,Cancelled,2019-11-02 07:22:00,NaT,Cancelled,False,False,"46.9674389764,9.55402859224",00:00:00,00:00:00
23547,2019-11-02,train,3260,IR13,False,False,8509004,Bad Ragaz,2019-11-02 07:26:00,NaT,Cancelled,2019-11-02 07:27:00,NaT,Cancelled,False,False,"47.0103066077,9.50521579767",00:00:00,00:00:00
23550,2019-11-02,train,3260,IR13,False,False,8509411,Sargans,2019-11-02 07:32:00,NaT,Cancelled,2019-11-02 07:36:00,2019-11-02 07:39:32,Okay,False,True,"47.0453635269,9.44539562754",00:03:32,00:00:00
20806,2019-11-02,train,3260,IR13,False,False,8509404,Buchs SG,2019-11-02 07:45:00,2019-11-02 07:47:59,Okay,2019-11-02 07:45:00,2019-11-02 07:48:54,Okay,False,True,"47.1684089962,9.47862968189",00:03:54,00:02:59
10899,2019-11-02,train,3260,IR13,False,False,8506319,Altstätten SG,2019-11-02 08:00:00,2019-11-02 08:02:28,Okay,2019-11-02 08:01:00,2019-11-02 08:03:23,Okay,False,False,"47.3742291734,9.55651096481",00:02:23,00:02:28
34011,2019-11-02,train,3260,IR13,False,False,8506317,Heerbrugg,2019-11-02 08:06:00,2019-11-02 08:07:31,Okay,2019-11-02 08:07:00,2019-11-02 08:08:52,Okay,False,False,"47.4103853071,9.62724352946",00:01:52,00:01:31
34013,2019-11-02,train,3260,IR13,False,False,8506314,St. Margrethen,2019-11-02 08:13:00,2019-11-02 08:12:50,Okay,2019-11-02 08:13:00,2019-11-02 08:13:41,Okay,False,False,"47.4532118157,9.63825154068",00:00:41,00:00:00
23553,2019-11-02,train,3260,IR13,False,False,8506311,Rorschach,2019-11-02 08:21:00,2019-11-02 08:20:23,Okay,2019-11-02 08:21:00,2019-11-02 08:23:09,Okay,False,False,"47.4779984016,9.50513168035",00:02:09,00:00:00
35130,2019-11-02,train,3260,IR13,False,False,8506302,St. Gallen,2019-11-02 08:35:00,2019-11-02 08:35:10,Okay,2019-11-02 08:37:00,NaT,Cancelled,False,False,"47.4231790668,9.36987712043",00:00:00,00:00:10


In [40]:
#now we create another DataFrame that is Line centric 
#needs the delay to be computed
dfline=df.sort_values(by='planned_departure_time').groupby("course_id").agg({'station_name':lambda x: list(x),'planned_arrival_time':\
    lambda x: list(x),'actual_arrival_time': lambda x: list(x),'cancelled_arrival':lambda x: list(x),'arrival_delayed': lambda x:\
     list(x),'actual_departure_time': lambda x: list(x),'planned_departure_time': lambda x: list(x),'cancelled_departure': lambda\
     x: list(x),'departure_delayed': lambda x: list(x),'geopos': lambda x:list(x),'departure_delay': lambda x: list(x),\
      'arrival_delay': lambda x: list(x)})

In [41]:
arri=[True in arr for arr in dfline.arrival_delayed]
depa=[True in arr for arr in dfline.departure_delayed]
dfline.arrival_delayed=arri
dfline.departure_delayed=depa

In [42]:
dfline.arrival_delay=dfline.arrival_delay.apply(max)
dfline.departure_delay=dfline.departure_delay.apply(max)

In [43]:
dfline.head(5)

,station_name,planned_arrival_time,actual_arrival_time,cancelled_arrival,arrival_delayed,actual_departure_time,planned_departure_time,cancelled_departure,departure_delayed,geopos,departure_delay,arrival_delay
course_id,,,,,,,,,,,,
3,"[Basel SBB, Zürich HB]","[NaT, 2019-11-02 09:00:00]","[NaT, 2019-11-02 09:00:34]","[Startpoint, Okay]",False,"[2019-11-02 08:08:54, NaT]","[2019-11-02 08:07:00, NaT]","[Okay, Endpoint]",False,"[47.5474041527,7.58955146721, 47.3781765756,8....",00:01:54,00:00:34
4,"[Zürich HB, Basel SBB, Basel Bad Bf]","[NaT, 2019-11-02 19:53:00, 2019-11-02 20:19:00]","[NaT, 2019-11-02 19:53:50, 2019-11-02 20:22:06]","[Startpoint, Okay, Okay]",True,"[2019-11-02 19:01:51, 2019-11-02 20:17:07, 201...","[2019-11-02 19:00:00, 2019-11-02 20:13:00, 201...","[Okay, Okay, Okay]",True,"[47.3781765756,8.54019221036, 47.5474041527,7....",00:04:07,00:03:06
5,"[Basel Bad Bf, Basel SBB]","[2019-11-02 09:34:00, 2019-11-02 09:47:00]","[2019-11-02 09:34:00, 2019-11-02 09:56:31]","[Okay, Okay]",True,"[2019-11-02 09:46:14, NaT]","[2019-11-02 09:40:00, NaT]","[Okay, Endpoint]",True,"[47.5681463635,7.60728432529, 47.5474041527,7....",00:06:14,00:09:31
6,"[Interlaken Ost, Interlaken West, Spiez, Thun,...","[NaT, 2019-11-02 12:03:00, 2019-11-02 12:21:00...","[NaT, 2019-11-02 12:04:33, 2019-11-02 12:21:09...","[Startpoint, Okay, Okay, Okay, Okay, Okay, Oka...",True,"[2019-11-02 12:00:39, 2019-11-02 12:05:54, 201...","[2019-11-02 12:00:00, 2019-11-02 12:05:00, 201...","[Okay, Okay, Okay, Okay, Okay, Okay, Okay, Oka...",True,"[46.6905054601,7.86900513744, 46.6826271766,7....",00:03:02,00:03:10
7,"[Basel Bad Bf, Basel SBB, Liestal, Olten, Bern...","[2019-11-02 13:22:00, 2019-11-02 13:30:00, 201...","[2019-11-02 13:22:00, 2019-11-02 13:44:06, 201...","[Okay, Okay, Okay, Okay, Okay, Okay, Okay, Oka...",True,"[2019-11-02 13:38:33, 2019-11-02 14:00:42, 201...","[2019-11-02 13:24:00, 2019-11-02 13:59:00, 201...","[Okay, Okay, Okay, Okay, Okay, Okay, Okay, Oka...",True,"[47.5681463635,7.60728432529, 47.5474041527,7....",00:14:33,00:14:06


In [44]:
dfline.loc[[("Startpoint" not in arr) for arr in dfline.cancelled_arrival]]

,station_name,planned_arrival_time,actual_arrival_time,cancelled_arrival,arrival_delayed,actual_departure_time,planned_departure_time,cancelled_departure,departure_delayed,geopos,departure_delay,arrival_delay
course_id,,,,,,,,,,,,
5,"[Basel Bad Bf, Basel SBB]","[2019-11-02 09:34:00, 2019-11-02 09:47:00]","[2019-11-02 09:34:00, 2019-11-02 09:56:31]","[Okay, Okay]",True,"[2019-11-02 09:46:14, NaT]","[2019-11-02 09:40:00, NaT]","[Okay, Endpoint]",True,"[47.5681463635,7.60728432529, 47.5474041527,7....",00:06:14,00:09:31
7,"[Basel Bad Bf, Basel SBB, Liestal, Olten, Bern...","[2019-11-02 13:22:00, 2019-11-02 13:30:00, 201...","[2019-11-02 13:22:00, 2019-11-02 13:44:06, 201...","[Okay, Okay, Okay, Okay, Okay, Okay, Okay, Oka...",True,"[2019-11-02 13:38:33, 2019-11-02 14:00:42, 201...","[2019-11-02 13:24:00, 2019-11-02 13:59:00, 201...","[Okay, Okay, Okay, Okay, Okay, Okay, Okay, Oka...",True,"[47.5681463635,7.60728432529, 47.5474041527,7....",00:14:33,00:14:06
9,"[Basel Bad Bf, Basel SBB, Zürich HB]","[2019-11-02 15:27:00, 2019-11-02 15:35:00, 201...","[2019-11-02 15:27:00, 2019-11-02 15:43:14, 201...","[Okay, Okay, Okay]",True,"[2019-11-02 15:34:03, 2019-11-02 16:08:57, NaT]","[2019-11-02 15:29:00, 2019-11-02 16:07:00, NaT]","[Okay, Okay, Endpoint]",True,"[47.5681463635,7.60728432529, 47.5474041527,7....",00:05:03,00:08:14
32,"[Brig, Sion, Montreux, Lausanne, Genève]","[2019-11-02 10:16:00, 2019-11-02 10:45:00, 201...","[2019-11-02 10:26:54, 2019-11-02 10:53:21, 201...","[Okay, Okay, Okay, Okay, Cancelled]",True,"[2019-11-02 10:29:03, 2019-11-02 10:54:53, 201...","[2019-11-02 10:20:00, 2019-11-02 10:47:00, 201...","[Okay, Okay, Okay, Cancelled, Endpoint]",True,"[46.3194210071,7.98809158467, 46.2275453615,7....",00:09:03,00:10:54
34,"[Brig, Sion, Montreux, Lausanne, Genève]","[2019-11-02 14:16:00, 2019-11-02 14:45:00, 201...","[2019-11-02 14:24:38, 2019-11-02 14:51:53, 201...","[Okay, Okay, Okay, Okay, Cancelled]",True,"[2019-11-02 14:27:19, 2019-11-02 14:54:02, 201...","[2019-11-02 14:20:00, 2019-11-02 14:47:00, 201...","[Okay, Okay, Okay, Cancelled, Endpoint]",True,"[46.3194210071,7.98809158467, 46.2275453615,7....",00:07:19,00:08:38
36,"[Brig, Sion, Montreux, Lausanne, Genève]","[2019-11-02 19:16:00, 2019-11-02 19:45:00, 201...","[2019-11-02 19:34:29, 2019-11-02 20:03:56, 201...","[Okay, Okay, Okay, Okay, Cancelled]",True,"[2019-11-02 19:37:28, 2019-11-02 20:10:19, 201...","[2019-11-02 19:18:00, 2019-11-02 19:47:00, 201...","[Okay, Okay, Okay, Cancelled, Endpoint]",True,"[46.3194210071,7.98809158467, 46.2275453615,7....",00:31:42,00:30:31
42,"[Brig, Sion, Montreux, Lausanne, Genève]","[2019-11-02 21:16:00, 2019-11-02 21:45:00, 201...","[2019-11-02 21:35:16, 2019-11-02 22:03:05, 201...","[Okay, Okay, Okay, Okay, Cancelled]",True,"[2019-11-02 21:38:07, 2019-11-02 22:06:14, 201...","[2019-11-02 21:18:00, 2019-11-02 21:47:00, 201...","[Okay, Okay, Okay, Cancelled, Endpoint]",True,"[46.3194210071,7.98809158467, 46.2275453615,7....",00:20:07,00:19:16
50,"[Brig, Visp, Spiez, Thun, Bern, Olten, Basel SBB]","[2019-11-02 09:16:00, 2019-11-02 09:25:00, 201...","[2019-11-02 09:24:16, 2019-11-02 09:32:16, 201...","[Okay, Okay, Okay, Okay, Okay, Okay, Okay]",True,"[2019-11-02 09:26:37, 2019-11-02 09:34:15, 201...","[2019-11-02 09:19:00, 2019-11-02 09:28:00, 201...","[Okay, Okay, Okay, Okay, Okay, Okay, Endpoint]",True,"[46.3194210071,7.98809158467, 46.2940248591,7....",00:07:37,00:08:16
52,"[Brig, Visp, Spiez, Thun, Bern, Olten, Basel S...","[2019-11-02 13:16:00, 2019-11-02 13:25:00, 201...","[2019-11-02 13:30:54, 2019-11-02 13:39:38, 201...","[Okay, Okay, Okay, Okay, Okay, Okay, Okay, Okay]",True,"[2019-11-02 13:33:33, 2019-11-02 13:41:21, 201...","[2019-11-02 13:19:00, 2019-11-02 13:28:00, 201...","[Okay, Okay, Okay, Okay, Okay, Okay, Okay, Okay]",True,"[46.3194210071,7.98809158467, 46.2940248591,7....",00:23:11,00:22:30


There is some lines that do not have an endpoint or startpoint. Those trains come from other countries or go into other contries. Since we focus on the Swiss train network we will drop those.

In [45]:
inddrop1=dfline.loc[[("Startpoint" not in arr) for arr in dfline.cancelled_arrival]].index
dfline.drop(inddrop1,inplace=True)
inddrop2=dfline.loc[[("Endpoint" not in arr) for arr in dfline.cancelled_departure]].index
dfline.drop(inddrop2,inplace=True)

In [46]:

def contain_or_not(list_of_words): #function to know wheter a line contains the stations of the user in the good order
    contains= [True,0]
    if station_departure in list_of_words:
        b=list_of_words[list_of_words.index(station_departure):]
        if station_arrival in b:
            contains= [True,list_of_words.index(station_departure)]
        else:
            contains= [False, None]
    else:
        contains=[False, None]
    
    return contains

def line_return(station_departure,station_arrival):
    f=dfline['station_name'].apply(contain_or_not)
    f=f.tolist()
    f2 = [item[0] for item in f]
    f3=[item[1] for item in f]#usefull to get the planned departure time and actual departure time
    f3= [incom for incom in f3 if str(incom) != 'None'] #might be useffull to drop na's
    g=dfline[f2]#list of all lines that contain departure and arrival station in the good order.
    departure_times_list=[]
    for i, j in enumerate(f3):
        departure_times_list.append(g['planned_departure_time'].iloc[i][j])
    return g,departure_times_list



In [47]:
#preprocess the data to be usefull in the model
df1=df[["station_name","planned_arrival_time","departure_delay","departure_delayed"]].dropna()
x=df1[["station_name","planned_arrival_time"]]
y=df1["departure_delay"]

x1=pd.get_dummies(data=x)
x1["hour"]=x["planned_arrival_time"].apply(lambda x: x.hour).astype(int)
x1["minute"]=x["planned_arrival_time"].apply(lambda x: x.minute).astype(int)
x1['day']=x['planned_arrival_time'].apply(lambda x: x.day).astype(int)
x2=x1.loc[:, x1.columns!="planned_arrival_time"]
y1=y.apply(lambda x: (x.seconds//60)%60).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x2, y1, test_size=0.3, random_state=1)

In [ ]:
def computeErrorRidgeRegCV(x, y, k, alpha):
    # Split the set in k-folds
    kf = KFold(n_splits = k)
    mse_k = []
    # For each configuration we compute the mean squared error
    for train_index, test_index in kf.split(x):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        reg = Ridge(alpha = alpha)
        reg.fit(x_train, y_train)
        predict = reg.predict(x_test)
        mse_k.append(mean_squared_error(y_test, predict))
    return mean(mse_k)
def computeCrossValidationErrors(x, y, k, alphas):
    cve=[]
    for alpha in alphas:
        cve.append(computeErrorRidgeRegCV(x, y, k, alpha))
    return cve
alphas = range(0,50)
cve = computeCrossValidationErrors(x_train, y_train, 5, alphas)

In [ ]:
import matplotlib.pyplot as plt


"""Plot the cross-validation error as a function of the regularization parameter alpha

Args:
    alphas (list of float): List of values for the regularization parameter.
    cve (list of float): List of errors corresponding to the alphas.
"""
def cross_validation_error_visualization(alphas, cve):
    plt.plot(alphas, cve, marker=".", color='b', label='train error')
    plt.xlabel("Alpha")
    plt.ylabel("Cross-validation error")
    plt.title("Cross-validation error evolution")
    plt.legend(loc=0)
    plt.grid(True)
    
cross_validation_error_visualization(alphas, cve)

In [ ]:
optimal_alpha = alphas[cve.index(min(cve))]
# Now we can use our test set and evaluate our model
model = Ridge(alpha = optimal_alpha)
model.fit(x_train, y_train)
predict = model.predict(x_test)
print('Optimal alpha: {}\nFinal model mean squared error: {}'
      .format(optimal_alpha, mean_squared_error(y_test, predict)))

In [ ]:
#we will try logistic regression to get probabilities of lateness so now the features will be the same and the output
#is a classifier departure_delayed
from sklearn.linear_model import LogisticRegression 
y=df1["departure_delayed"]
x_train, x_test, y_train, y_test = train_test_split(x2, y, test_size=0.3, random_state=1)
lr=LogisticRegression(random_state=0)
model=lr.fit(x_train,y_train)
model.score(x_test,y_test)

In [ ]:
model.predict_proba(x_test)

In [ ]:
#interactiv part 
from difflib import SequenceMatcher
from datetime import *
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
l=df.station_name.unique()
station=input("enter the start station:")
list=[]
if ~(station in l):
    for name in l:
        list.append(similar(station,name))
    station=l[list.index(max(list))]
time=input("enter the time(format 00:00:00):")
time= datetime.strptime(time, '%H::%M::%S').time()

## Delay Analysis

We had columns to store the delay of departure and arrival in order to make stats on it later.
We consider that a train that doesn't have a scheduled start, hasn't departure delay.  
Similarly, a train that arrives early has no negative delay, we set it to 0.

In [ ]:
#Compute the delay of the departure
df['departure_delay'] = df['actual_departure_time'] - df['planned_departure_time']
#Set at 0 the delay when the train left in advance its station
df.loc[df.departure_delay < datetime.timedelta(0), 'departure_delay'] = datetime.timedelta(0)
#Replace the NaT values to 0
df.departure_delay.fillna(datetime.timedelta(0), inplace=True)


#Compute the delay of the arrival
df['arrival_delay'] = df['actual_arrival_time'] - df['planned_arrival_time']
#Set at 0 the delay when the train arrived in advance
df.loc[df.arrival_delay < datetime.timedelta(0), 'arrival_delay'] = datetime.timedelta(0)
#Replace the NaT values to 0£
df.arrival_delay.fillna(datetime.timedelta(0), inplace=True)

* What's the average time of delay?
* What's the max?

In [ ]:
df['departure_delay'].mean().min
print("\tDeparture delay - average: {0}/max: {1}\n\tArrival delay - average: {2}/max: {3}"
     .format(df['departure_delay'].mean(), df['departure_delay'].max(),
                 df['arrival_delay'].mean(), df['arrival_delay'].max()))

* What is the most common type of delay
* What's the proportion of departure delayed train?

In [ ]:
dep_delayed = df.groupby(pd.Grouper(key = 'departure_delay', freq = '60s')).size()

In [ ]:
dep_delayed.plot()
plt.xlabel("Delay")
plt.ylabel("Number of delayed train")
plt.title("Delay distribution")
plt.yscale('log')
plt.show()

Even with a log scale, it's seems that most of the delayed trains are less than 3 minutes late.
Since his type of delay is not too penalizing for the users we choose to redefine the limit of the delay.  
*We now consider that a train is delayed from the moment it's over 3 min late.*

In [ ]:
dep_delayed.loc[dep_delayed.index > datetime.timedelta(minutes=3)].plot()
plt.xlabel("Delay")
plt.ylabel("Number of delayed train")
plt.title("Delay distribution (>3min)")
plt.show()

We see that the delays are still mainly stuck between 3 minutes and 15 minutes.

In [ ]:
dep_delayed.loc[(dep_delayed.index > datetime.timedelta(minutes=3)) \
                & (dep_delayed.index < datetime.timedelta(minutes=15))].plot()
plt.xlabel("Delay")
plt.ylabel("Number of delayed train")
plt.title("Delay distribution (>3min and <15min)")
plt.show()

In [ ]:
# Compute the proportion of delayed train compare to other trains
proportion = dep_delayed.loc[(dep_delayed.index > datetime.timedelta(minutes=3))].sum() / dep_delayed.sum()
print('Proportion of delayed train (>3 min): {} %'.format(round(proportion * 100,2)))

* What time of the day are trains more likely to be late ?

In [ ]:
late = df.loc[df['departure_delay'] > datetime.timedelta(minutes=3), ['actual_departure_time']]
times = pd.DatetimeIndex(late['actual_departure_time'])
grouped = late.groupby([times.hour]).count()
grouped.plot(kind='bar', legend=None)
plt.xlabel("Hour of the day")
plt.ylabel("Number of delayed train")
plt.title("Delay distribution depending on the time of day")
plt.show()

There is a decrease in the number of delays in the period from midnight to 4am.  
Except this slot and the leaving work hour (~6pm), delays seem to have an uniform distribution.

* Does a late departing train necessarily have a late arrival delay? (Is there a strong correlation between depature and arrival delay?)

In [ ]:
print('Correlation between departure and arrival delay: {}'.format(df['departure_delayed'].corr(df['arrival_delayed'])))

We observe that, even if the train is delayed at the beginning it's not necessarily late to the arrival. 

## Geographic data plotting test

Since we want to plot some geograpic data from our dataset (network...), we tried with our subdataset.

In [ ]:
# We create two new column from the geopos column
df['latitude'], df['longitude'] = df['geopos'].str.split(',', 1).str

# Type conversion
df['latitude'] = df['latitude'].astype('float')
df['longitude'] = df['longitude'].astype('float')

# We compute the area that will include all spatial points
BBox = (df.longitude.min(), df.longitude.max(), df.latitude.min(), df.latitude.max())
print('Geograpic plotting boundaries: ({0}, {2}) and ({1}, {3})'.format(BBox[0], BBox[1], BBox[2], BBox[3])) 

We use this data to extract the corresponding area from: https://www.openstreetmap.org/  
We will now be able to plot our data.

In [ ]:
# Retrieve the image map
map = plt.imread('data/test_map.png')

# Create the figure and plot our data
fig, ax = plt.subplots(figsize = (15,7))
ax.scatter(df.longitude, df.latitude, s=10)
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.imshow(map, extent = BBox)
ax.set_title('SBB Train station location')
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.show()

Now we will try to plot the whole network by using our line centered dataset.  
We create two new columns in the dataset to store the list of longitudes and latitudes for each line of the dataset:

In [ ]:
all_lat = []
all_long = []
# For each row, we retrieve the longitude and latitude of the line (list)
for i, row in dfline.iterrows():
    longitudes = []
    latitudes = []
    for pos in row['geopos']:
        long, lat = pos.split(',')
        # We need to convert str to float for the plotting part
        longitudes.append(float(long))
        latitudes.append(float(lat))

    all_long.append(longitudes)
    all_lat.append(latitudes)

dfline['longitudes'] = all_long
dfline['latitudes'] = all_lat

Now we are able to plot the whole network:

In [ ]:
# Retrieve the image map
map = plt.imread('data/test_map.png')

# Create the figure and plot our data
fig, ax = plt.subplots(figsize = (15,7))

for i, row in dfline.iterrows():
    ax.scatter(row['longitudes'], row['latitudes'], s=10)
    plt.plot(row['longitudes'], row['latitudes'])

ax.imshow(map, extent = BBox)
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.set_title('SBB Train station location')
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.show()


*Some line trajectory seem quite strange but it's just because the train is a direct one. We only have the departure and arrival station so we can't represent the real trajectory.*  
Now we can also consider the delay for each line and observe the delay repartition through the network:

# Plan for the Milestone 3

## Dataset

For the first analysis we only used a subset of the available data. For the next milestone we will use all the data that we managed to get during this second milestone.

## Delay analysis

We have already started doing an analysis on this topic. For the next step, we would like to analyze more deeply the geographical part (which stations are the most affected ...).So we will be able to answer the question defined in our readme => *'In which stations the delays are the most frequent?'*

## Delay reasons analysis

We found a nice data set from which we can extract information on what causes delays. The goal will be to get the "big picture" on what causes the delays and maybe tell where the CFF company should focus their efforts to reduce the amount and impact of the delays. 

## Visualisation

We will improve the visualisation of the network. Adding edges, reflecting the size of traffic on different edges and nodes by some visual aspect (thickness, color, shape). Making it iteractive like seeing stats when hovering over a certain node or edge and zooming or filtering out the little stations. We will try different tools like Folium which seems adeqaute for this task.

## Report

We will mainly focus on doing a nice data story following the narrative of "How to efficiently take trains in Switzerland and what to expect". We will certainly use Jekyll for our data story and so we need to further our knowledge of that tool.

## Things that we dropped

The database that we found on the frequency of passengers is not usefull at all as it consists of just frequency of passenger per week for each station and doesn't give much more information.
We also dropped the effect of weather on the delay as it would be too much for this project.
We won't be comparing the SBB with other country trains system as it would mean to much data exploration and data scrapping.